# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
import sqlite3
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponseData.db')
df = pd.read_sql('SELECT * FROM "disaster_response"', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def load_data():
    
    """
    Loading transformed data of Disaster Response and split into X and y.
  
    This function connects to the sqlite server where the ETL-Disaster Response Data is stored, loads the dataset, 
    removes the column "id" and splits the remaining columns into messages (X) and number values (y).
      
    Parameters:
    None
  
    Returns:
    array (X, y): messages and number values.
  
    """
    
    engine = create_engine('sqlite:///DisasterResponseData.db')
    df = pd.read_sql('SELECT * FROM "disaster_response"', engine)
    df.drop(columns=['id'], inplace=True)
    X = df.message.values
    y = df.select_dtypes(include=np.number).values
    category_names = np.unique(y)
    return X, y, category_names

In [5]:
X, y, category_names = load_data()

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    """
    Transforming string text into arrays of words.
  
    This function splits strings into words, removes stopwords and lemmatize the words.
      
    Parameters:
    text (string): string input containing text.
  
    Returns:
    lemmed (array): array with separate text strings for machine learning model input.
  
    """
    
    tokens = word_tokenize(text.lower().strip())
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokens]
    return lemmed

In [7]:
tokenize(X[0])

['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
def model_pipeline():
    """
    Machine learning pipeline to predict classification.
  
    This function contains a pipeline where the data input gets vectorized, transformed and the output gets predicted with
    KNeighboursClassifier.
      
    Parameters:
    None.
  
    Returns:
    Pipeline.
  
    """
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
    return pipeline
pipeline = model_pipeline()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X,y, category_names = load_data()
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [10]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
def display_results(y_test, y_pred):
    
    """
    Displaying model results.
  
    This function displays the f1 score, precision and recall in the classification report of the predicted output.
    It shows how well the selected prediction model works.
      
    Parameters:
    y_test: actual results from the dataset, created within train_test_split.
    y_pred: predicted results in Model.
  
    Returns:
    classification report: shows precision, recall, f1-score and support on different target names.
      
    """
    
    type_of_target(y_test)
    type_of_target(y_pred)
    target_names = category_names
    report = classification_report(np.hstack(y_test),np.hstack(y_pred))
    
    print('Target names:', target_names)
    print('Classification report:\n', report)

display_results(y_test, y_pred)

Target names: [0 1 2]
Classification report:
              precision    recall  f1-score   support

          0       0.94      0.99      0.97    215097
          1       0.75      0.40      0.53     20796
          2       0.65      0.25      0.37        51

avg / total       0.93      0.94      0.93    235944



### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f22ccb2e1e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_fea

In [13]:
parameters = {
    'clf__estimator__leaf_size': [20, 30, 40],
}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = True)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__leaf_size': [20, 30, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print("\nBest Parameters:", cv.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 95.6min finished



Best Parameters: {'clf__estimator__leaf_size': 20}


In [15]:
display_results(y_test, y_pred)

Target names: [0 1 2]
Classification report:
              precision    recall  f1-score   support

          0       0.94      0.99      0.96    215232
          1       0.78      0.34      0.47     20651
          2       0.83      0.16      0.27        61

avg / total       0.93      0.93      0.92    235944



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
def model_pipeline_rfc():
    
    """
    Machine learning pipeline to predict classification.
  
    This function contains a pipeline where the data input gets vectorized, transformed and the output gets predicted with
    RandomForestClassifier.
      
    Parameters:
    None.
  
    Returns:
    Pipeline.
  
    """
    
    pipeline = Pipeline([
            ('vec', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
    return pipeline

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f22ccb2e1e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_fea

In [13]:
def model_pipeline_rfc():
    
    """
    Machine learning pipeline with GridSearch to predict classification.
  
    This function contains a pipeline where the data input gets vectorized, transformed and the output gets predicted with
    RandomForestClassifier. The pipeline gets modified by GridSearch to look for the best parameter values.
      
    Parameters:
    None.
  
    Returns:
    Pipeline.
  
    """
    pipeline = Pipeline([
            ('vec', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
    
    parameters = {'clf__estimator__max_features':['sqrt', 0.5]}

    cv = GridSearchCV(pipeline, param_grid = parameters, cv = 5, n_jobs = 6, verbose = True)

    return cv
cv_rfc = model_pipeline_rfc()

In [ ]:
def model_pipeline_rfc():
    
    """
    Machine learning pipeline with GridSearch to predict classification.
  
    This function contains a pipeline where the data input gets vectorized, transformed and the output gets predicted with
    RandomForestClassifier. The pipeline gets modified by GridSearch to look for the best parameter values.
      
    Parameters:
    None.
  
    Returns:
    Pipeline.
  
    """
    pipeline = Pipeline([
            ('vec', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
    
    parameters = {'clf__estimator__max_features':['sqrt', 0.5],
                  'clf__estimator__n_estimators':[20, 50]}

    cv = GridSearchCV(pipeline, param_grid = parameters, cv = 5, n_jobs = 6, verbose = True)

    return cv
cv_rfc = model_pipeline_rfc()

In [14]:
cv_rfc.fit(X_train, y_train)
y_pred_rfc = cv_rfc.predict(X_test)
print("\nBest Parameters:", cv_rfc.best_params_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed: 108.9min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed: 108.9min finished



Best Parameters: {'clf__estimator__max_features': 0.5}


In [15]:
display_results(y_test, y_pred_rfc)

Target names: [0 1 2]
Classification report:
              precision    recall  f1-score   support

          0       0.96      0.98      0.97    214688
          1       0.76      0.59      0.67     21217
          2       0.60      0.15      0.24        39

avg / total       0.94      0.95      0.94    235944



### 9. Export your model as a pickle file

In [14]:
final_model = model_pipeline_rfc()
filename = 'finalized_model.sav'
pickle.dump(final_model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.